In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob, cv2
from PIL import Image
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# # 데이터를 불러오기
# X_loaded = np.load('/content/drive/MyDrive/2.양추정/np.array_2_이미지수량동일하게/X_data_612.npy')
# y_loaded = np.load('/content/drive/MyDrive/2.양추정/np.array_2_이미지수량동일하게/y_data_612.npy')

# print("X_loaded shape:", X_loaded.shape)
# print("y_loaded shape:", y_loaded.shape)

X_loaded shape: (16600, 224, 224, 3)
y_loaded shape: (16600,)


In [ ]:
# c = X_loaded/255.0
# X_type = c.astype(np.float32)

# np.save('/content/drive/MyDrive/2.양추정/np.array_2_이미지수량동일하게/X_data_612_new.npy', X_type)

In [11]:
X = np.load('/content/drive/MyDrive/2.양추정/np.array_2_이미지수량동일하게/X_data_612_new.npy')
y = np.load('/content/drive/MyDrive/2.양추정/np.array_2_이미지수량동일하게/y_data_612.npy')

print("X_new_loaded shape:", X_new_loaded.shape)
print("y_loaded shape:", y_loaded.shape)

X_new_loaded shape: (16600, 224, 224, 3)
y_loaded shape: (16600,)


In [12]:
X[0]

array([[[0.9411765 , 0.91764706, 0.92156863],
        [0.9411765 , 0.91764706, 0.92156863],
        [0.9411765 , 0.91764706, 0.92156863],
        ...,
        [0.9490196 , 0.92941177, 0.93333334],
        [0.9490196 , 0.92941177, 0.93333334],
        [0.9490196 , 0.92941177, 0.93333334]],

       [[0.9411765 , 0.91764706, 0.92156863],
        [0.9411765 , 0.91764706, 0.92156863],
        [0.9411765 , 0.91764706, 0.92156863],
        ...,
        [0.9490196 , 0.92941177, 0.93333334],
        [0.9490196 , 0.92941177, 0.93333334],
        [0.9490196 , 0.92941177, 0.93333334]],

       [[0.9411765 , 0.91764706, 0.92156863],
        [0.9411765 , 0.91764706, 0.92156863],
        [0.9411765 , 0.91764706, 0.92156863],
        ...,
        [0.9490196 , 0.92941177, 0.93333334],
        [0.9490196 , 0.92941177, 0.93333334],
        [0.9490196 , 0.92941177, 0.93333334]],

       ...,

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0

In [13]:
#라벨(1~5) 확인하기
a = np.unique(y)
a

array([1, 2, 3, 4, 5])

In [14]:
#원-핫 인코딩
yy = pd.get_dummies(y)
yy

,1,2,3,4,5
0,True,False,False,False,False
1,True,False,False,False,False
2,True,False,False,False,False
3,True,False,False,False,False
4,True,False,False,False,False
...,...,...,...,...,...
16595,False,False,False,False,True
16596,False,False,False,False,True
16597,False,False,False,False,True
16598,False,False,False,False,True


In [9]:
# #train 데이터 크기: (13280, 224, 224, 3) (13280, 5)
# #test 데이터 크기: (3320, 224, 224, 3) (3320, 5)

# X_train, X_test, y_train, y_test = train_test_split(X_new_loaded, y, test_size=0.2, shuffle=True, stratify=y, random_state=42)

# print("train 데이터 크기:", X_train.shape, y_train.shape)
# print("test 데이터 크기:", X_test.shape, y_test.shape)

train 데이터 크기: (13280, 224, 224, 3) (13280, 5)
test 데이터 크기: (3320, 224, 224, 3) (3320, 5)


In [17]:
#shuffle 제외하고 진행
X_train, X_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, stratify=y, random_state=42)

print("train 데이터 크기:", X_train.shape, y_train.shape)
print("test 데이터 크기:", X_test.shape, y_test.shape)

train 데이터 크기: (13280, 224, 224, 3) (13280, 5)
test 데이터 크기: (3320, 224, 224, 3) (3320, 5)


In [ ]:
#shuffle = False
X_train, X_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, shuffle=False, stratify=y, random_state=42)

print("train 데이터 크기:", X_train.shape, y_train.shape)
print("test 데이터 크기:", X_test.shape, y_test.shape)

In [ ]:
#model-18
#X데이터 /255 정규화 진행함

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

#layer Conv2D 추가
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # 과적합 방지

model.add(Dense(5, activation='softmax'))

model.summary()

checkpoint_path = "/content/drive/MyDrive/YANG/model/checkpoint/model_checkpoint_model6-18.h5"
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

model.fit(X_train, y_train, validation_split=0.25, epochs=1000, batch_size=42, verbose=1, callbacks=[checkpoint, early_stopping])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 conv2d_4 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 conv2d_5 (Conv2D)           (None, 218, 218, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 109, 109, 128)     0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 109, 109, 128)     0         
                                                                 
 flatten_1 (Flatten)         (None, 1520768)           0         
                                                      

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


238/238 [==============================] - 112s 334ms/step - loss: 2.3148 - accuracy: 0.1947 - val_loss: 1.6095 - val_accuracy: 0.1943
Epoch 2/1000
238/238 [==============================] - ETA: 0s - loss: 1.6101 - accuracy: 0.2002
Epoch 2: val_loss did not improve from 1.60951
238/238 [==============================] - 63s 264ms/step - loss: 1.6101 - accuracy: 0.2002 - val_loss: 1.6096 - val_accuracy: 0.1976
Epoch 3/1000
238/238 [==============================] - ETA: 0s - loss: 1.6119 - accuracy: 0.2060
Epoch 3: val_loss did not improve from 1.60951
238/238 [==============================] - 63s 264ms/step - loss: 1.6119 - accuracy: 0.2060 - val_loss: 1.6105 - val_accuracy: 0.1970
Epoch 4/1000
238/238 [==============================] - ETA: 0s - loss: 1.5996 - accuracy: 0.2396
Epoch 4: val_loss did not improve from 1.60951
238/238 [==============================] - 63s 264ms/step - loss: 1.5996 - accuracy: 0.2396 - val_loss: 1.6141 - val_accuracy: 0.2045
Epoch 5/1000
238/238 [======

KeyboardInterrupt: 

In [ ]:
#model-19 (/255) 정규화 진행함

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Dropout(0.25))

#layer Conv2D 추가
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.6))

model.add(Dense(5, activation='softmax'))

model.summary()

checkpoint_path = "/content/drive/MyDrive/YANG/model/checkpoint/model_checkpoint_model6-19.h5"
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

model.fit(X_train, y_train, validation_split=0.25, epochs=1000, batch_size=42, verbose=1, callbacks=[checkpoint, early_stopping])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 dropout (Dropout)           (None, 222, 222, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 dropout_1 (Dropout)         (None, 220, 220, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 218, 218, 128)     73856     
                                                                 
 max_pooling2d (MaxPooling2  (None, 109, 109, 128)     0         
 D)                                                              
                                                        

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


238/238 [==============================] - 1081s 5s/step - loss: 4.6831 - accuracy: 0.1966 - val_loss: 1.6095 - val_accuracy: 0.1976
Epoch 2/1000
238/238 [==============================] - ETA: 0s - loss: 1.6096 - accuracy: 0.1981
Epoch 2: val_loss did not improve from 1.60947
238/238 [==============================] - 1032s 4s/step - loss: 1.6096 - accuracy: 0.1981 - val_loss: 1.6096 - val_accuracy: 0.1976
Epoch 3/1000


KeyboardInterrupt: 

In [ ]:
#model-20
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Dropout(0.25))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(Dropout(0.25))

model.add(Conv2D(32, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(5, activation='softmax'))

model.summary()

checkpoint_path = "/content/drive/MyDrive/YANG/model/checkpoint/model_checkpoint_model6-20.h5"
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

history = model.fit(X_train, y_train, validation_split=0.25, epochs=1000, batch_size=42, verbose=1, callbacks=[checkpoint, early_stopping])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 220, 220, 32)      2432      
                                                                 
 dropout_9 (Dropout)         (None, 220, 220, 32)      0         
                                                                 
 conv2d_7 (Conv2D)           (None, 218, 218, 32)      9248      
                                                                 
 dropout_10 (Dropout)        (None, 218, 218, 32)      0         
                                                                 
 conv2d_8 (Conv2D)           (None, 217, 217, 32)      4128      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 108, 108, 32)      0         
 g2D)                                                            
                                                      

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


238/238 [==============================] - 323s 1s/step - loss: 1.7966 - accuracy: 0.1999 - val_loss: 1.6097 - val_accuracy: 0.1961
Epoch 2/1000
238/238 [==============================] - ETA: 0s - loss: 1.6095 - accuracy: 0.1991
Epoch 2: val_loss improved from 1.60966 to 1.60965, saving model to /content/drive/MyDrive/YANG/model/checkpoint/model_checkpoint_model6-20.h5
238/238 [==============================] - 320s 1s/step - loss: 1.6095 - accuracy: 0.1991 - val_loss: 1.6096 - val_accuracy: 0.1943
Epoch 3/1000
238/238 [==============================] - ETA: 0s - loss: 1.6095 - accuracy: 0.2019
Epoch 3: val_loss did not improve from 1.60965
238/238 [==============================] - 323s 1s/step - loss: 1.6095 - accuracy: 0.2019 - val_loss: 1.6097 - val_accuracy: 0.1943
Epoch 4/1000
238/238 [==============================] - ETA: 0s - loss: 1.6095 - accuracy: 0.1991
Epoch 4: val_loss did not improve from 1.60965
238/238 [==============================] - 322s 1s/step - loss: 1.6095 - 

KeyboardInterrupt: 

In [ ]:
#model-21
#모델 설정
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(Dropout(0.25))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(Dropout(0.25))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(5, activation='softmax'))

model.summary()

checkpoint_path = "/content/drive/MyDrive/YANG/model/checkpoint/model_checkpoint_model6-21.h5"
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

history = model.fit(X_train, y_train, validation_split=0.25, epochs=1000, batch_size=42, verbose=1, callbacks=[checkpoint, early_stopping])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 dropout_13 (Dropout)        (None, 222, 222, 32)      0         
                                                                 
 conv2d_10 (Conv2D)          (None, 220, 220, 32)      9248      
                                                                 
 dropout_14 (Dropout)        (None, 220, 220, 32)      0         
                                                                 
 conv2d_11 (Conv2D)          (None, 218, 218, 32)      9248      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 109, 109, 32)      0         
 g2D)                                                            
                                                      

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


238/238 [==============================] - 358s 1s/step - loss: 1.7579 - accuracy: 0.1958 - val_loss: 1.6096 - val_accuracy: 0.1940
Epoch 2/1000
238/238 [==============================] - ETA: 0s - loss: 1.6096 - accuracy: 0.1958
Epoch 2: val_loss did not improve from 1.60955
238/238 [==============================] - 357s 1s/step - loss: 1.6096 - accuracy: 0.1958 - val_loss: 1.6096 - val_accuracy: 0.1943
Epoch 3/1000
238/238 [==============================] - ETA: 0s - loss: 1.6096 - accuracy: 0.1962
Epoch 3: val_loss did not improve from 1.60955
238/238 [==============================] - 356s 1s/step - loss: 1.6096 - accuracy: 0.1962 - val_loss: 1.6096 - val_accuracy: 0.1940
Epoch 4/1000
238/238 [==============================] - ETA: 0s - loss: 1.6095 - accuracy: 0.1936
Epoch 4: val_loss did not improve from 1.60955
238/238 [==============================] - 356s 1s/step - loss: 1.6095 - accuracy: 0.1936 - val_loss: 1.6097 - val_accuracy: 0.1940
Epoch 5/1000
  4/238 [...............

KeyboardInterrupt: 

In [ ]:
#model-22
#모델 설정
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(5, activation='softmax'))

model.summary()

checkpoint_path = "/content/drive/MyDrive/YANG/model/checkpoint/model_checkpoint_model6-22.h5"
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

history = model.fit(X_train, y_train, validation_split=0.25, epochs=1000, batch_size=42, verbose=1, callbacks=[checkpoint, early_stopping])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 conv2d_13 (Conv2D)          (None, 220, 220, 32)      9248      
                                                                 
 conv2d_14 (Conv2D)          (None, 218, 218, 32)      9248      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 109, 109, 32)      0         
 g2D)                                                            
                                                                 
 dropout_17 (Dropout)        (None, 109, 109, 32)      0         
                                                                 
 flatten_4 (Flatten)         (None, 380192)            0         
                                                      

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


238/238 [==============================] - 317s 1s/step - loss: 1.6658 - accuracy: 0.2042 - val_loss: 1.6097 - val_accuracy: 0.1895
Epoch 2/1000
238/238 [==============================] - ETA: 0s - loss: 1.6099 - accuracy: 0.1947
Epoch 2: val_loss did not improve from 1.60966
238/238 [==============================] - 313s 1s/step - loss: 1.6099 - accuracy: 0.1947 - val_loss: 1.6097 - val_accuracy: 0.1943
Epoch 3/1000
238/238 [==============================] - ETA: 0s - loss: 1.6090 - accuracy: 0.2137
Epoch 3: val_loss did not improve from 1.60966
238/238 [==============================] - 316s 1s/step - loss: 1.6090 - accuracy: 0.2137 - val_loss: 1.6124 - val_accuracy: 0.1804
Epoch 4/1000
238/238 [==============================] - ETA: 0s - loss: 1.6079 - accuracy: 0.2192
Epoch 4: val_loss did not improve from 1.60966
238/238 [==============================] - 316s 1s/step - loss: 1.6079 - accuracy: 0.2192 - val_loss: 1.6100 - val_accuracy: 0.2000
Epoch 5/1000
238/238 [===============



---

집

In [ ]:
#model-22-1 :
#과적합 Dropout 크기 변경해서 다시 한번
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(5, activation='softmax'))

model.summary()

checkpoint_path = "/content/drive/MyDrive/YANG/model/checkpoint/model_checkpoint_model6-22-1.h5"
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

history = model.fit(X_train, y_train, validation_split=0.25, epochs=1000, batch_size=42, verbose=1, callbacks=[checkpoint, early_stopping])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 218, 218, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 109, 109, 32)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 109, 109, 32)      0         
                                                                 
 flatten (Flatten)           (None, 380192)            0         
                                                        

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


238/238 [==============================] - 321s 1s/step - loss: 1.7128 - accuracy: 0.1986 - val_loss: 1.6096 - val_accuracy: 0.1943
Epoch 2/1000
238/238 [==============================] - ETA: 0s - loss: 1.6095 - accuracy: 0.1963
Epoch 2: val_loss did not improve from 1.60955
238/238 [==============================] - 312s 1s/step - loss: 1.6095 - accuracy: 0.1963 - val_loss: 1.6096 - val_accuracy: 0.1943
Epoch 3/1000
238/238 [==============================] - ETA: 0s - loss: 1.6096 - accuracy: 0.1940
Epoch 3: val_loss did not improve from 1.60955
238/238 [==============================] - 312s 1s/step - loss: 1.6096 - accuracy: 0.1940 - val_loss: 1.6097 - val_accuracy: 0.1943
Epoch 4/1000
238/238 [==============================] - ETA: 0s - loss: 1.6095 - accuracy: 0.1991
Epoch 4: val_loss did not improve from 1.60955
238/238 [==============================] - 313s 1s/step - loss: 1.6095 - accuracy: 0.1991 - val_loss: 1.6097 - val_accuracy: 0.1976
Epoch 5/1000
238/238 [===============

KeyboardInterrupt: 

In [ ]:
#model-23
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(5, activation='softmax'))

model.summary()

checkpoint_path = "/content/drive/MyDrive/YANG/model/checkpoint/model_checkpoint_model6-23.h5"
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

history = model.fit(X_train, y_train, validation_split=0.25, epochs=1000, batch_size=42, verbose=1, callbacks=[checkpoint, early_stopping])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 conv2d_4 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 conv2d_5 (Conv2D)           (None, 218, 218, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 109, 109, 64)      0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 109, 109, 64)      0         
                                                                 
 flatten_1 (Flatten)         (None, 760384)            0         
                                                      

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


238/238 [==============================] - 506s 2s/step - loss: 1.6505 - accuracy: 0.1999 - val_loss: 1.6096 - val_accuracy: 0.1961
Epoch 2/1000
238/238 [==============================] - ETA: 0s - loss: 1.6095 - accuracy: 0.1978
Epoch 2: val_loss did not improve from 1.60958
238/238 [==============================] - 503s 2s/step - loss: 1.6095 - accuracy: 0.1978 - val_loss: 1.6097 - val_accuracy: 0.1961
Epoch 3/1000
238/238 [==============================] - ETA: 0s - loss: 1.6095 - accuracy: 0.1919
Epoch 3: val_loss did not improve from 1.60958
238/238 [==============================] - 501s 2s/step - loss: 1.6095 - accuracy: 0.1919 - val_loss: 1.6098 - val_accuracy: 0.1943
Epoch 4/1000
238/238 [==============================] - ETA: 0s - loss: 1.6095 - accuracy: 0.2006
Epoch 4: val_loss did not improve from 1.60958
238/238 [==============================] - 504s 2s/step - loss: 1.6095 - accuracy: 0.2006 - val_loss: 1.6097 - val_accuracy: 0.1943
Epoch 5/1000
238/238 [===============

KeyboardInterrupt: 

In [ ]:
#model-24
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.summary()
checkpoint_path = "/content/drive/MyDrive/YANG/model/checkpoint/model_checkpoint_model6-24.h5"
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

history = model.fit(X_train, y_train, validation_split=0.25, epochs=1000, batch_size=42, verbose=1, callbacks=[checkpoint, early_stopping])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 conv2d_7 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 110, 110, 64)      0         
 g2D)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 110, 110, 64)      0         
                                                                 
 flatten_2 (Flatten)         (None, 774400)            0         
                                                                 
 dense_4 (Dense)             (None, 32)                24780832  
                                                      

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


238/238 [==============================] - 268s 1s/step - loss: 2.0851 - accuracy: 0.1952 - val_loss: 1.6095 - val_accuracy: 0.1976
Epoch 2/1000
238/238 [==============================] - ETA: 0s - loss: 1.6095 - accuracy: 0.1965
Epoch 2: val_loss did not improve from 1.60948
238/238 [==============================] - 263s 1s/step - loss: 1.6095 - accuracy: 0.1965 - val_loss: 1.6096 - val_accuracy: 0.1943
Epoch 3/1000
238/238 [==============================] - ETA: 0s - loss: 1.6096 - accuracy: 0.1953
Epoch 3: val_loss did not improve from 1.60948
238/238 [==============================] - 265s 1s/step - loss: 1.6096 - accuracy: 0.1953 - val_loss: 1.6097 - val_accuracy: 0.1943
Epoch 4/1000
238/238 [==============================] - ETA: 0s - loss: 1.6095 - accuracy: 0.2015
Epoch 4: val_loss did not improve from 1.60948
238/238 [==============================] - 268s 1s/step - loss: 1.6095 - accuracy: 0.2015 - val_loss: 1.6098 - val_accuracy: 0.1961
Epoch 5/1000
238/238 [===============

KeyboardInterrupt: 

In [10]:
#model-24-1
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.summary()
checkpoint_path = "/content/drive/MyDrive/YANG/model/checkpoint/model_checkpoint_model6-24-1.h5"
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

history = model.fit(X_train, y_train, validation_split=0.25, epochs=1000, batch_size=42, verbose=1, callbacks=[checkpoint, early_stopping])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 111, 111, 32)      0         
                                                                 
 flatten (Flatten)           (None, 394272)            0         
                                                                 
 dense (Dense)               (None, 32)                12616736  
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                        

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


238/238 [==============================] - 73s 304ms/step - loss: 1.9445 - accuracy: 0.1971 - val_loss: 1.6096 - val_accuracy: 0.1943
Epoch 2/1000
238/238 [==============================] - ETA: 0s - loss: 1.6121 - accuracy: 0.1967
Epoch 2: val_loss did not improve from 1.60958
238/238 [==============================] - 68s 286ms/step - loss: 1.6121 - accuracy: 0.1967 - val_loss: 1.6097 - val_accuracy: 0.1943
Epoch 3/1000
238/238 [==============================] - ETA: 0s - loss: 1.6096 - accuracy: 0.1981
Epoch 3: val_loss did not improve from 1.60958
238/238 [==============================] - 68s 285ms/step - loss: 1.6096 - accuracy: 0.1981 - val_loss: 1.6097 - val_accuracy: 0.1943
Epoch 4/1000
238/238 [==============================] - ETA: 0s - loss: 1.6096 - accuracy: 0.1948
Epoch 4: val_loss did not improve from 1.60958
238/238 [==============================] - 68s 284ms/step - loss: 1.6096 - accuracy: 0.1948 - val_loss: 1.6097 - val_accuracy: 0.1943
Epoch 5/1000
238/238 [=======

In [18]:
#train_test_split 에서 shuffle 제외하고 과적합 되었던 모델 다시 실행
#model-22-2
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(5, activation='softmax'))

model.summary()

checkpoint_path = "/content/drive/MyDrive/YANG/model/checkpoint/model_checkpoint_model6-22-2.h5"
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

history = model.fit(X_train, y_train, validation_split=0.25, epochs=1000, batch_size=42, verbose=1, callbacks=[checkpoint, early_stopping])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 conv2d_2 (Conv2D)           (None, 220, 220, 32)      9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 218, 218, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 109, 109, 32)      0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 109, 109, 32)      0         
                                                                 
 flatten_1 (Flatten)         (None, 380192)            0         
                                                      

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


238/238 [==============================] - 321s 1s/step - loss: 1.6983 - accuracy: 0.1997 - val_loss: 1.6094 - val_accuracy: 0.1952
Epoch 2/1000
238/238 [==============================] - ETA: 0s - loss: 1.6090 - accuracy: 0.2092
Epoch 2: val_loss did not improve from 1.60940
238/238 [==============================] - 316s 1s/step - loss: 1.6090 - accuracy: 0.2092 - val_loss: 1.6098 - val_accuracy: 0.1967
Epoch 3/1000
238/238 [==============================] - ETA: 0s - loss: 1.6089 - accuracy: 0.2160
Epoch 3: val_loss did not improve from 1.60940
238/238 [==============================] - 318s 1s/step - loss: 1.6089 - accuracy: 0.2160 - val_loss: 1.6149 - val_accuracy: 0.2051
Epoch 4/1000
238/238 [==============================] - ETA: 0s - loss: 1.5998 - accuracy: 0.2432
Epoch 4: val_loss did not improve from 1.60940
238/238 [==============================] - 315s 1s/step - loss: 1.5998 - accuracy: 0.2432 - val_loss: 1.6109 - val_accuracy: 0.2133
Epoch 5/1000
238/238 [===============

KeyboardInterrupt: 

In [19]:
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

104/104 [==============================] - 12s 117ms/step - loss: 1.6713 - accuracy: 0.2473
Test loss: 1.6712538003921509
Test accuracy: 0.24728915095329285


In [20]:
#shuffle = False
X_train, X_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, shuffle=False, stratify=y, random_state=42)

print("train 데이터 크기:", X_train.shape, y_train.shape)
print("test 데이터 크기:", X_test.shape, y_test.shape)

ValueError: Stratified train/test split is not implemented for shuffle=False

In [ ]:
#train_test_split 에서 shuffle=false, 과적합 되었던 모델 다시 실행
#model-22-3
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(5, activation='softmax'))

model.summary()

checkpoint_path = "/content/drive/MyDrive/YANG/model/checkpoint/model_checkpoint_model6-22-3.h5"
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

history = model.fit(X_train, y_train, validation_split=0.25, epochs=1000, batch_size=42, verbose=1, callbacks=[checkpoint, early_stopping])

score = model.evaluate(X_test, y_test, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])